
# **CPU AvatarArtist4: Diffusion Transformer Training**

https://kumapowerliu.github.io/AvatarArtist/

---

## **Step 4 Pipeline Explanation: DiT (Diffusion Transformer) Training**

## Overview
This pipeline trains a **Diffusion Transformer (DiT)** model that learns to generate 3D triplanes from 2D avatar images. Unlike Step 2 (which trained a GAN), this uses **diffusion models** - a newer, more stable approach that progressively denoises random noise into structured 3D representations.

---

## Key Concept: Diffusion Models

### **What is Diffusion?**

```
Training:
  Clean Triplane → Add Noise (1000 steps) → Random Noise
  Model learns to reverse this process

Generation:
  Random Noise → Remove Noise (1000 steps) → Clean Triplane
  Conditioned on input image to control the output
```

**Analogy**: Like watching a video of ink dissolving in water in reverse - the model learns to un-dissolve the ink back into a clear pattern.

---

## Pipeline Architecture

### **1. Data Loading (TriplaneDataset)**

```python
TriplaneDataset(
    data_dir="./dit_training_data",  # From Step 3
    load_images=True,
    image_size=256,
    triplane_size=256
)
```

**Loads:**
- **Images**: 256×256 RGB images (condition)
- **Triplanes**: Static + Dynamic features (target to predict)
- **Metadata**: Links images to triplanes
- **3DMM params**: Shape, expression, pose (optional)

**Data Structure:**
```python
batch = {
    'image': [B, 3, 256, 256],      # Condition
    'triplane': [B, 96, 256, 256],  # Target (67 static + 29 dynamic)
    'static': [B, 67, 256, 256],
    'dynamic': [B, 29, 256, 256],
    'z': [B, 512],
    'shape': [B, 80],
    'exp': [B, 64],
    'pose': [B, 6]
}
```

---

### **2. Model Architecture (DiffusionTransformer)**

#### **A) Input Processing**

**PatchEmbed**: Converts images into patches
```python
Image [B, 3, 256, 256] 
  → Patches [B, 256, 768]  # 16×16 patches, 768-dim embeddings
```

**Two Parallel Embedders:**
1. **x_embedder**: Embeds noisy triplane (what we're denoising)
2. **c_embedder**: Embeds condition image (guides denoising)

#### **B) Timestep Embedding**

```python
TimestepEmbedding(dim=768)
```
- Encodes diffusion timestep (0-1000) into embedding
- Tells model "how noisy is the input?"
- Uses sinusoidal encoding (like Transformers)

#### **C) Transformer Blocks (DiTBlock)**

```python
DiTBlock × depth (6 or 12 layers)
  ├─ Adaptive Layer Norm (AdaLN)
  ├─ Multi-Head Self-Attention
  └─ Feed-Forward Network (MLP)
```

**Key Innovation: AdaLN (Adaptive Layer Norm)**
- Conditions each layer on the timestep
- Learns different transformations for different noise levels
- 6 parameters per block: shift/scale/gate for attention & MLP

**Formula:**
```
h = LayerNorm(x)
h = h * (1 + scale) + shift
output = x + gate * Attention(h)
```

#### **D) Output Layer (FinalLayer)**

```python
FinalLayer
  ├─ Adaptive Layer Norm
  ├─ Linear projection
  └─ Unpatchify: Patches → Image
```

**Output**: Predicted noise [B, 96, 256, 256]

---

### **3. Diffusion Process (GaussianDiffusion)**

#### **Forward Process (Training)**

```python
q_sample(x_start, t, noise):
  x_t = sqrt(α_t) * x_0 + sqrt(1 - α_t) * ε
```

**Steps:**
1. Take clean triplane `x_0`
2. Sample random timestep `t` ∈ [0, 1000]
3. Add Gaussian noise according to schedule
4. Result: Noisy triplane `x_t`

**Noise Schedule:**
```python
β: 0.0001 → 0.02  # Linear schedule
α = 1 - β
ᾱ = cumprod(α)     # Cumulative product
```

#### **Reverse Process (Inference)**

```python
p_sample(x_t, t, condition):
  ε_predicted = model(x_t, t, condition)
  x_{t-1} = (x_t - √(1-ᾱ) * ε) / √ᾱ  # Remove noise
```

**Generation Loop:**
```
for t in [1000, 999, ..., 1, 0]:
    x = remove_noise(x, t)  # Progressive denoising
return x  # Clean triplane
```

---

## Training Process

### **4. Loss Function**

**Simple MSE Loss:**
```python
loss = MSE(predicted_noise, actual_noise)
```

**Why this works:**
- Model learns to predict noise at any timestep
- Once it can predict noise, it can reverse the diffusion
- Simpler than GAN losses (no discriminator needed)

**Training Step:**
```
1. Load batch: (image, triplane)
2. Sample random timestep t
3. Add noise to triplane: x_t = q_sample(triplane, t)
4. Predict noise: ε_pred = model(x_t, t, image)
5. Compute loss: L = MSE(ε_pred, ε_actual)
6. Backpropagate and update weights
```

---

## Parameter Settings

### **Model Configurations**

**Small Model (Recommended for storage constraints):**
```python
hidden_size = 384
depth = 6
num_heads = 6
Parameters: ~40M
Checkpoint size: ~160MB
```

**Standard Model (Better quality):**
```python
hidden_size = 768
depth = 12
num_heads = 12
Parameters: ~172M
Checkpoint size: ~700MB
```

### **Training Hyperparameters**

| Parameter | Default | Purpose | Tuning Guide |
|-----------|---------|---------|--------------|
| `batch_size` | 4 | Samples per iteration | GPU 24GB: 4-8, 12GB: 2 |
| `num_epochs` | 100 | Training epochs | More data = fewer epochs |
| `lr` | 1e-4 | Learning rate | Stable, rarely needs tuning |
| `weight_decay` | 0.0 | L2 regularization | Increase if overfitting |
| `timesteps` | 1000 | Diffusion steps | More = better quality, slower |
| `save_every` | 10 | Checkpoint frequency | Adjust for storage |
| `keep_last_n` | 2 | Recent checkpoints to keep | Saves disk space |

### **Diffusion Hyperparameters**

| Parameter | Value | Meaning |
|-----------|-------|---------|
| `beta_start` | 0.0001 | Noise at t=0 (very small) |
| `beta_end` | 0.02 | Noise at t=1000 (large) |
| `timesteps` | 1000 | Number of diffusion steps |

---

## Training Workflow

### **Initialization Phase**

```
1. Load dataset (90% train, 10% validation)
2. Initialize model (Small or Standard)
3. Initialize diffusion schedule
4. Create optimizer (AdamW)
5. Create learning rate scheduler (Cosine)
```

### **Training Loop**

```python
for epoch in range(num_epochs):
    # Training
    for batch in train_loader:
        loss = train_step(batch)
    
    # Validation
    val_loss = validate(val_loader)
    
    # Learning rate decay
    scheduler.step()
    
    # Save checkpoints
    if epoch % save_every == 0:
        save_checkpoint()
    
    # Save best model
    if val_loss < best_val_loss:
        save_best_model()
```

### **Memory Optimization Features**

1. **Checkpoint Cleanup**: Keeps only last 2 checkpoints
2. **Best Model Tracking**: Saves model with lowest validation loss
3. **Gradient Clipping**: Prevents exploding gradients (max norm = 1.0)
4. **Mixed Precision**: Can be enabled for faster training (not shown in code)

---

## Expected Results

### **Training Metrics**

#### **Healthy Training Signs:**

```
Epoch 1-10 (Early):
  Train Loss: 0.15 - 0.08 (decreasing rapidly)
  Val Loss: 0.12 - 0.09
  LR: 1e-4

Epoch 10-50 (Mid):
  Train Loss: 0.08 - 0.04 (steady decrease)
  Val Loss: 0.09 - 0.05
  LR: 5e-5 (cosine decay)

Epoch 50-100 (Late):
  Train Loss: 0.04 - 0.02 (slow decrease)
  Val Loss: 0.05 - 0.03
  LR: 1e-6 (near minimum)
```

#### **Convergence Indicators:**

✅ **Good Signs:**
- Train loss steadily decreases
- Val loss decreases similarly (small gap)
- Loss stabilizes around 0.02-0.05
- No sudden spikes or NaN

❌ **Problem Signs:**
- Val loss >> Train loss (overfitting)
- Loss not decreasing after 20 epochs (learning rate too low)
- NaN or Inf values (numerical instability)
- Loss oscillating wildly (learning rate too high)

### **Loss Interpretation**

| Loss Range | Meaning | Action |
|------------|---------|--------|
| 0.01 - 0.03 | Excellent | Model converged well |
| 0.03 - 0.06 | Good | Acceptable quality |
| 0.06 - 0.10 | Moderate | May need more training |
| > 0.10 | Poor | Check data/model/hyperparameters |

---

## Output Files

### **Checkpoint Structure**

```
dit_checkpoints/
├── checkpoint_0010.pt    # Epoch 10 (will be deleted)
├── checkpoint_0020.pt    # Epoch 20 (will be deleted)
├── ...
├── checkpoint_0090.pt    # Epoch 90 (kept - last 2)
├── checkpoint_0100.pt    # Epoch 100 (kept - last 2)
├── best_model.pt         # Best validation loss
└── final_model.pt        # Final epoch
```

**Checkpoint Contents:**
```python
{
    'epoch': 100,
    'model': model.state_dict(),      # Model weights
    'optimizer': optimizer.state_dict(),  # Optimizer state
    'scheduler': scheduler.state_dict()   # LR scheduler state
}
```

**File Sizes:**
- Small model: ~160 MB per checkpoint
- Standard model: ~700 MB per checkpoint

---

## What Results Indicate

### **1. Training Curves**

**Ideal Curve Shape:**
```
Loss
  |
  |  Train ----
  |            ----
  |                ----___
  |  Val ----           ----___
  |         ----                ----___
  |             ----                   ----___
  +-----------------------------------------> Epoch
```

**Interpretation:**
- **Parallel curves** = Good generalization
- **Diverging curves** = Overfitting (val loss increases)
- **Flat curve** = Underfitting (model too simple or LR too low)

### **2. Model Quality Assessment**

#### **Direct Indicators:**

| Metric | Good | Moderate | Poor |
|--------|------|----------|------|
| Final Train Loss | < 0.03 | 0.03-0.06 | > 0.06 |
| Final Val Loss | < 0.05 | 0.05-0.08 | > 0.08 |
| Train-Val Gap | < 0.02 | 0.02-0.04 | > 0.04 |
| Convergence Speed | < 50 epochs | 50-80 epochs | > 80 epochs |

#### **Generation Quality (After Training):**

To evaluate model quality, generate samples:

```python
# Load best model
model = load_model('dit_checkpoints/best_model.pt')

# Sample from diffusion
condition_image = load_image('test.png')
triplane = diffusion.sample(model, shape, condition_image)

# Check quality:
- Triplane values in reasonable range [-5, 5]
- No NaN or Inf values
- Smooth spatial features (no checkerboard patterns)
- Good correlation with condition image
```

---

## Common Issues & Solutions

### **Problem 1: High Validation Loss**

**Symptoms:**
```
Train Loss: 0.02
Val Loss: 0.15  # Large gap
```

**Causes & Solutions:**
1. **Overfitting**
   - Add weight decay: `weight_decay=0.01`
   - Reduce model size: Use Small model
   - Get more training data (Step 3)
   - Add dropout (modify DiTBlock)

2. **Train/Val distribution mismatch**
   - Check data split is random
   - Ensure validation set is representative

### **Problem 2: Slow Convergence**

**Symptoms:**
```
Epoch 50: Loss still > 0.10
```

**Solutions:**
1. Increase learning rate: `lr=2e-4`
2. Warm-up learning rate schedule
3. Increase model capacity: Use Standard model
4. Check data quality (Step 3)
5. Increase batch size (if memory allows)

### **Problem 3: Loss Spikes**

**Symptoms:**
```
Epoch 30: Loss=0.05
Epoch 31: Loss=0.15  # Sudden spike
Epoch 32: Loss=0.08
```

**Solutions:**
1. Reduce learning rate: `lr=5e-5`
2. Enable gradient clipping (already implemented)
3. Check for corrupted data samples
4. Reduce batch size

### **Problem 4: Out of Memory**

**Symptoms:**
```
RuntimeError: CUDA out of memory
```

**Solutions:**
1. Reduce batch size: `batch_size=2`
2. Use Small model
3. Reduce image/triplane size: `256 → 128`
4. Enable gradient checkpointing (advanced)
5. Use CPU (very slow)

### **Problem 5: Disk Space Shortage**

**Symptoms:**
```
OSError: [Errno 28] No space left on device
```

**Solutions:**
1. Set `keep_last_n=1` (keep only 1 checkpoint)
2. Use Small model (~160MB vs 700MB)
3. Increase `save_every` to 20 or 50
4. Delete old checkpoints manually
5. Save only best model, skip periodic saves

---

## After Training: Inference Usage

### **Load Trained Model**

```python
# Initialize model
model = DiffusionTransformer(
    img_size=256,
    patch_size=16,
    in_channels=96,
    condition_channels=3,
    hidden_size=384,  # Match training config
    depth=6,
    num_heads=6
)

# Load weights
checkpoint = torch.load('dit_checkpoints/best_model.pt')
model.load_state_dict(checkpoint['model'])
model.eval()
```

### **Generate Triplane from Image**

```python
# Load condition image
image = load_and_preprocess_image('avatar.png')  # [1, 3, 256, 256]

# Initialize diffusion
diffusion = GaussianDiffusion(timesteps=1000, device='cuda')

# Sample triplane
triplane = diffusion.sample(
    model=model,
    shape=(1, 96, 256, 256),  # Batch, Channels, H, W
    condition=image
)

# triplane is now a 3D representation of the avatar
# Can be rendered from different angles using volume rendering
```

### **Controllable Generation**

```python
# Modify expression by adjusting dynamic features
triplane_static = triplane[:, :67]   # Identity (unchanged)
triplane_dynamic = triplane[:, 67:]  # Expression (modify this)

# Example: Amplify expression
triplane_dynamic_amplified = triplane_dynamic * 1.5

# Reconstruct
triplane_new = torch.cat([triplane_static, triplane_dynamic_amplified], dim=1)

# Render to get new image with stronger expression
```

---

## Comparison: Step 2 (GAN) vs Step 4 (DiT)

| Aspect | Step 2 (Next3D GAN) | Step 4 (DiT) |
|--------|---------------------|--------------|
| **Model Type** | GAN (Generator + Discriminator) | Diffusion (Denoising Model) |
| **Training Stability** | Moderate (requires ADA, R1, careful tuning) | High (simpler loss, no discriminator) |
| **Generation Quality** | Good (fast generation) | Better (slower generation) |
| **Controllability** | Limited (via 3DMM params) | High (via condition image + triplane decomposition) |
| **Training Speed** | Faster per epoch | Slower per epoch |
| **Inference Speed** | Fast (1 forward pass) | Slow (1000 denoising steps) |
| **Use Case** | Generate varied avatars quickly | High-quality, controlled generation |

---

## Performance Expectations

### **Training Time**

| Hardware | Batch Size | Time/Epoch | 100 Epochs | Small vs Standard |
|----------|-----------|------------|------------|-------------------|
| CPU (8 cores) | 1 | 60+ min | 100+ hours | N/A (too slow) |
| GPU (GTX 1080) | 2 | 5-8 min | 8-13 hours | Small recommended |
| GPU (RTX 3090) | 4-8 | 2-4 min | 3-7 hours | Either works |
| GPU (A100) | 8-16 | 1-2 min | 2-3 hours | Standard recommended |

**With 1000 training samples:**
- Small model, GPU (RTX 3090), batch=4: ~250 iterations/epoch → ~3 min/epoch → ~5 hours total

### **Generation Time (Inference)**

| Diffusion Steps | Quality | Time (RTX 3090) | Use Case |
|-----------------|---------|-----------------|----------|
| 50 | Preview | 1-2 sec | Fast preview |
| 250 | Good | 5-8 sec | Balanced |
| 1000 | Best | 15-20 sec | Final output |

**Trade-off**: More steps = better quality but slower

---

## Summary

This pipeline:

✅ **Trains** a Diffusion Transformer to generate 3D triplanes  
✅ **Conditions** on 2D images for controlled generation  
✅ **Uses** stable diffusion training (no GAN instability)  
✅ **Enables** high-quality, controllable 3D-aware generation  
✅ **Optimizes** disk space with checkpoint management  
✅ **Provides** small and standard model options

**Key Innovation**: Combines Transformers (powerful learning) + Diffusion (stable training) + 3D triplane decomposition (controllability)

**Result Interpretation:**
- **Low loss (< 0.05)** = Model learned to denoise triplanes well
- **Small train-val gap** = Good generalization
- **Stable training** = No sudden spikes or divergence
- **Quality samples** = Triplanes can render high-quality 3D avatars

**Next Steps**: Use trained model to generate novel 3D avatars from 2D reference images, with control over expressions and poses through triplane manipulation.

---

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate
!pip install controlnet-aux opencv-python pillow
!pip install mediapipe==0.10.13

In [ ]:
dit_training_data = '/kaggle/input/cpu-avatarartist3-triplane-decomposition/dit_training_data'
dit_checkpoints = '/kaggle/input/cpu-avatarartist2-next3d-4d-gan-fine-tuning/next3d_checkpoints'
final_model_path = '/kaggle/input/cpu-avatarartist2-next3d-4d-gan-fine-tuning/next3d_checkpoints/final_model.pt'


In [ ]:
"""
Diffusion Transformer (DiT) Training Script
Training a Diffusion model to predict 3D triplanes from 2D images.

Reference: DiT (Scalable Diffusion Models with Transformers)
https://arxiv.org/abs/2212.09748
"""

import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from tqdm import tqdm
import json
import torchvision.transforms as transforms
from einops import rearrange, repeat
import shutil


# ==================== Dataloader ====================

class TriplaneDataset(Dataset):
    """Dataset for Image and Triplane pairs"""
    
    def __init__(
        self,
        data_dir: str,
        load_images: bool = True,
        image_size: int = 256,
        triplane_size: int = 256
    ):
        """
        Args:
            data_dir: Dataset directory (dit_training_data)
            load_images: Whether to load images
            image_size: Target image size
            triplane_size: Target triplane size
        """
        self.data_dir = Path(data_dir)
        self.load_images = load_images
        self.image_size = image_size
        self.triplane_size = triplane_size
        
        # Load metadata
        with open(self.data_dir / 'metadata.json', 'r') as f:
            self.metadata = json.load(f)
        
        # Load dataset info
        with open(self.data_dir / 'dataset_info.json', 'r') as f:
            self.info = json.load(f)
        
        # Image transformations
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        
        print(f"Dataset loaded: {len(self.metadata)} samples")
        print(f"Static channels: {self.info['static_channels']}")
        print(f"Dynamic channels: {self.info['dynamic_channels']}")
        
        # Path test: Check if the first sample can be loaded
        if len(self.metadata) > 0:
            test_meta = self.metadata[0]
            test_triplane_path = test_meta['triplane_path']
            
            # Path conversion test
            if not os.path.isabs(test_triplane_path):
                if test_triplane_path.startswith('./'):
                    test_triplane_path = test_triplane_path[2:]
                if test_triplane_path.startswith('dit_training_data/'):
                    test_triplane_path = test_triplane_path.replace('dit_training_data/', '', 1)
                test_triplane_path = os.path.join(self.data_dir, test_triplane_path)
            
            if os.path.exists(test_triplane_path):
                print(f"✓ Path test passed: First sample accessible")
            else:
                print(f"⚠ Path test failed:")
                print(f"  Original: {test_meta['triplane_path']}")
                print(f"  Converted: {test_triplane_path}")
                print(f"  Exists: {os.path.exists(test_triplane_path)}")
                
                # Display directory contents for debugging
                triplane_dir = os.path.join(self.data_dir, 'triplanes')
                if os.path.exists(triplane_dir):
                    files = sorted(os.listdir(triplane_dir))[:5]
                    print(f"  First 5 files in triplanes/: {files}")
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        meta = self.metadata[idx]
        
        # Path correction: Convert relative path to absolute
        triplane_path = meta['triplane_path']
        
        if not os.path.isabs(triplane_path):
            # Convert './dit_training_data/triplanes/...' -> 'triplanes/...'
            if triplane_path.startswith('./'):
                triplane_path = triplane_path[2:]
            
            # Remove 'dit_training_data/' prefix if present
            if triplane_path.startswith('dit_training_data/'):
                triplane_path = triplane_path.replace('dit_training_data/', '', 1)
            
            # Combine with data_dir
            triplane_path = os.path.join(self.data_dir, triplane_path)
        
        # Load triplane
        if triplane_path.endswith('.npz'):
            data = np.load(triplane_path)
            triplane_data = {
                k: torch.from_numpy(data[k]).float() for k in data.keys()
            }
        else:  # .pth
            triplane_data = torch.load(triplane_path)
        
        # Resize triplanes if necessary
        static = triplane_data['static']
        dynamic = triplane_data['dynamic']
        
        if static.shape[-1] != self.triplane_size:
            static = F.interpolate(
                static.unsqueeze(0),
                size=(self.triplane_size, self.triplane_size),
                mode='bilinear',
                align_corners=False
            ).squeeze(0)
            
            dynamic = F.interpolate(
                dynamic.unsqueeze(0),
                size=(self.triplane_size, self.triplane_size),
                mode='bilinear',
                align_corners=False
            ).squeeze(0)
        
        # Concatenate triplanes
        triplane = torch.cat([static, dynamic], dim=0)
        
        result = {
            'triplane': triplane,  # [C, H, W]
            'static': static,
            'dynamic': dynamic,
            'z': triplane_data['z'],
            'shape': triplane_data['shape'],
            'exp': triplane_data['exp'],
            'pose': triplane_data['pose'],
        }
        
        # Load image
        if self.load_images and meta.get('image_path'):
            image_path = meta['image_path']
            
            # Correct image path similarly
            if not os.path.isabs(image_path):
                if image_path.startswith('./'):
                    image_path = image_path[2:]
                if image_path.startswith('dit_training_data/'):
                    image_path = image_path.replace('dit_training_data/', '', 1)
                image_path = os.path.join(self.data_dir, image_path)
            
            if os.path.exists(image_path):
                image = Image.open(image_path).convert('RGB')
                image = self.transform(image)
                result['image'] = image
            else:
                print(f"⚠ Image not found: {image_path}")
                # Dummy image as fallback
                result['image'] = torch.zeros(3, self.image_size, self.image_size)
        
        return result


# ==================== DiT Model Components ====================

class TimestepEmbedding(nn.Module):
    """Sinusoidal Embedding for Timesteps"""
    
    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.SiLU(),
            nn.Linear(dim * 4, dim)
        )
    
    def forward(self, timesteps):
        """
        Args:
            timesteps: [B] timesteps
        Returns:
            [B, dim] embedding vector
        """
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=timesteps.device) * -emb)
        emb = timesteps[:, None] * emb[None, :]
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        
        return self.mlp(emb)


class PatchEmbed(nn.Module):
    """Splits images into patches and embeds them"""
    
    def __init__(self, img_size=256, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        
        self.proj = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )
    
    def forward(self, x):
        """
        Args:
            x: [B, C, H, W]
        Returns:
            [B, N, D] where N=num_patches, D=embed_dim
        """
        x = self.proj(x)  # [B, D, H', W']
        x = rearrange(x, 'b d h w -> b (h w) d')
        return x


class DiTBlock(nn.Module):
    """
    DiT Transformer Block with Adaptive Layer Norm (AdaLN)
    Uses conditional Layer Norm.
    """
    
    def __init__(self, dim: int, num_heads: int, mlp_ratio: float = 4.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim, elementwise_affine=False)
        self.attn = nn.MultiheadAttention(dim, num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim, elementwise_affine=False)
        
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Linear(mlp_hidden_dim, dim)
        )
        
        # Adaptive layer norm parameters
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(dim, 6 * dim)
        )
    
    def forward(self, x, c):
        """
        Args:
            x: [B, N, D] Tokens
            c: [B, D] Condition vector (e.g., time embedding)
        Returns:
            [B, N, D]
        """
        # Extract AdaLN parameters
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = \
            self.adaLN_modulation(c).chunk(6, dim=-1)
        
        # Attention with adaptive LN
        h = self.norm1(x)
        h = h * (1 + scale_msa.unsqueeze(1)) + shift_msa.unsqueeze(1)
        h, _ = self.attn(h, h, h)
        x = x + gate_msa.unsqueeze(1) * h
        
        # MLP with adaptive LN
        h = self.norm2(x)
        h = h * (1 + scale_mlp.unsqueeze(1)) + shift_mlp.unsqueeze(1)
        h = self.mlp(h)
        x = x + gate_mlp.unsqueeze(1) * h
        
        return x


class FinalLayer(nn.Module):
    """Final Output Layer"""
    
    def __init__(self, hidden_size: int, patch_size: int, out_channels: int):
        super().__init__()
        self.norm_final = nn.LayerNorm(hidden_size, elementwise_affine=False)
        self.linear = nn.Linear(hidden_size, patch_size * patch_size * out_channels)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 2 * hidden_size)
        )
    
    def forward(self, x, c):
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=-1)
        x = self.norm_final(x) * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)
        x = self.linear(x)
        return x


class DiffusionTransformer(nn.Module):
    """
    Diffusion Transformer (DiT)
    Predicts 3D triplane conditioned on a 2D image.
    """
    
    def __init__(
        self,
        img_size: int = 256,
        patch_size: int = 16,
        in_channels: int = 96,  # Number of channels in triplane
        condition_channels: int = 3,  # Number of channels in condition image
        hidden_size: int = 768,
        depth: int = 12,
        num_heads: int = 12,
        mlp_ratio: float = 4.0
    ):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.out_channels = in_channels
        self.num_patches = (img_size // patch_size) ** 2
        
        # Input embedding (Noisy triplane)
        self.x_embedder = PatchEmbed(
            img_size, patch_size, in_channels, hidden_size
        )
        
        # Condition embedding (Image)
        self.c_embedder = PatchEmbed(
            img_size, patch_size, condition_channels, hidden_size
        )
        
        # Time embedding
        self.t_embedder = TimestepEmbedding(hidden_size)
        
        # Positional embedding
        self.pos_embed = nn.Parameter(
            torch.zeros(1, self.num_patches, hidden_size)
        )
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            DiTBlock(hidden_size, num_heads, mlp_ratio)
            for _ in range(depth)
        ])
        
        # Final layer
        self.final_layer = FinalLayer(hidden_size, patch_size, self.out_channels)
        
        self.initialize_weights()
    
    def initialize_weights(self):
        # Weight initialization
        def _basic_init(module):
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        self.apply(_basic_init)
        
        # Positional embedding
        nn.init.normal_(self.pos_embed, std=0.02)
        
        # Zero-initialize output layers
        nn.init.constant_(self.final_layer.linear.weight, 0)
        nn.init.constant_(self.final_layer.linear.bias, 0)
    
    def unpatchify(self, x):
        """
        Args:
            x: [B, N, patch_size^2 * C]
        Returns:
            [B, C, H, W]
        """
        p = self.patch_size
        h = w = int(x.shape[1] ** 0.5)
        x = rearrange(x, 'b (h w) (p1 p2 c) -> b c (h p1) (w p2)',
                     h=h, p1=p, p2=p)
        return x
    
    def forward(self, x, t, c):
        """
        Args:
            x: [B, C, H, W] Noisy triplane
            t: [B] Timestep
            c: [B, 3, H, W] Condition image
        Returns:
            [B, C, H, W] Predicted noise
        """
        # Embeddings
        x = self.x_embedder(x) + self.pos_embed  # [B, N, D]
        c_embed = self.c_embedder(c)             # [B, N, D]
        t_embed = self.t_embedder(t)             # [B, D]
        
        # Combine conditions
        x = x + c_embed  # Add image condition
        c = t_embed      # Time condition via AdaLN
        
        # Transformer blocks
        for block in self.blocks:
            x = block(x, c)
        
        # Final layer
        x = self.final_layer(x, c)
        x = self.unpatchify(x)
        
        return x


# ==================== Diffusion Process ====================

class GaussianDiffusion:
    """DDPM Gaussian Diffusion Process"""
    
    def __init__(
        self,
        timesteps: int = 1000,
        beta_start: float = 0.0001,
        beta_end: float = 0.02,
        device: str = 'cuda'
    ):
        self.timesteps = timesteps
        self.device = device
        
        # Beta schedule (linear)
        self.betas = torch.linspace(beta_start, beta_end, timesteps).to(device)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        self.alphas_cumprod_prev = F.pad(self.alphas_cumprod[:-1], (1, 0), value=1.0)
        
        # Calculations for diffusion q(x_t | x_{t-1})
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - self.alphas_cumprod)
        
        # Calculations for posterior q(x_{t-1} | x_t, x_0)
        self.posterior_variance = (
            self.betas * (1.0 - self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        )
    
    def q_sample(self, x_start, t, noise=None):
        """
        Forward diffusion: x_0 -> x_t
        """
        if noise is None:
            noise = torch.randn_like(x_start)
        
        sqrt_alphas_cumprod_t = self.sqrt_alphas_cumprod[t]
        sqrt_one_minus_alphas_cumprod_t = self.sqrt_one_minus_alphas_cumprod[t]
        
        # Broadcast to match x_start shape
        sqrt_alphas_cumprod_t = sqrt_alphas_cumprod_t[:, None, None, None]
        sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod_t[:, None, None, None]
        
        return (
            sqrt_alphas_cumprod_t * x_start +
            sqrt_one_minus_alphas_cumprod_t * noise
        )
    
    def p_losses(self, model, x_start, t, condition, noise=None):
        """
        Training loss calculation
        """
        if noise is None:
            noise = torch.randn_like(x_start)
        
        # Forward diffusion
        x_noisy = self.q_sample(x_start, t, noise)
        
        # Predict noise
        predicted_noise = model(x_noisy, t, condition)
        
        # MSE loss
        loss = F.mse_loss(predicted_noise, noise)
        
        return loss
    
    @torch.no_grad()
    def p_sample(self, model, x, t, condition):
        """
        Reverse diffusion: x_t -> x_{t-1}
        """
        predicted_noise = model(x, t, condition)
        
        # Coefficients
        alpha = self.alphas[t][:, None, None, None]
        alpha_cumprod = self.alphas_cumprod[t][:, None, None, None]
        beta = self.betas[t][:, None, None, None]
        
        # Predict x_0
        pred_x0 = (x - torch.sqrt(1 - alpha_cumprod) * predicted_noise) / torch.sqrt(alpha_cumprod)
        
        # Calculate x_{t-1}
        if t[0] > 0:
            noise = torch.randn_like(x)
            sigma = torch.sqrt(self.posterior_variance[t])[:, None, None, None]
        else:
            noise = 0
            sigma = 0
        
        mean = (1 / torch.sqrt(alpha)) * (
            x - (beta / torch.sqrt(1 - alpha_cumprod)) * predicted_noise
        )
        
        return mean + sigma * noise
    
    @torch.no_grad()
    def sample(self, model, shape, condition):
        """
        Sampling: Generate clean data from random noise
        """
        batch_size = shape[0]
        device = condition.device
        
        # Start from random noise
        x = torch.randn(shape, device=device)
        
        # Reverse diffusion steps
        for i in tqdm(reversed(range(self.timesteps)), desc='Sampling', total=self.timesteps):
            t = torch.full((batch_size,), i, device=device, dtype=torch.long)
            x = self.p_sample(model, x, t, condition)
        
        return x


# ==================== Training Loop ====================

class DiTTrainer:
    """Diffusion Transformer Trainer"""
    
    def __init__(
        self,
        model: nn.Module,
        diffusion: GaussianDiffusion,
        device: str = 'cuda',
        lr: float = 1e-4,
        weight_decay: float = 0.0
    ):
        self.model = model.to(device)
        self.diffusion = diffusion
        self.device = device
        
        # Optimizer
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
            betas=(0.9, 0.999)
        )
        
        # Learning rate scheduler
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=1000, eta_min=1e-6
        )
        
        print(f"Trainer initialized")
        print(f"  Parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"  Device: {device}")
    
    def train_step(self, batch: Dict[str, torch.Tensor]):
        """Single training step"""
        self.model.train()
        self.optimizer.zero_grad()
        
        # Data
        triplane = batch['triplane'].to(self.device)
        image = batch['image'].to(self.device)
        
        batch_size = triplane.shape[0]
        
        # Random timesteps
        t = torch.randint(
            0, self.diffusion.timesteps,
            (batch_size,),
            device=self.device
        ).long()
        
        # Loss calculation
        loss = self.diffusion.p_losses(self.model, triplane, t, image)
        
        # Backpropagation
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
        
        self.optimizer.step()
        
        return {'loss': loss.item()}
    
    @torch.no_grad()
    def validate(self, dataloader):
        """Validation step"""
        self.model.eval()
        total_loss = 0
        
        for batch in dataloader:
            triplane = batch['triplane'].to(self.device)
            image = batch['image'].to(self.device)
            
            batch_size = triplane.shape[0]
            t = torch.randint(
                0, self.diffusion.timesteps,
                (batch_size,),
                device=self.device
            ).long()
            
            loss = self.diffusion.p_losses(self.model, triplane, t, image)
            total_loss += loss.item()
        
        return total_loss / len(dataloader)
    
    def save_checkpoint(self, path: str, epoch: int, keep_last_n: int = 2):
        """
        Saves checkpoint (Storage-optimized version)
        
        Args:
            path: Save path
            epoch: Current epoch
            keep_last_n: Number of recent checkpoints to keep (older ones are deleted)
        """
        checkpoint = {
            'epoch': epoch,
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
        }
        
        torch.save(checkpoint, path)
        print(f"Checkpoint saved: {path}")
        
        # Display file size
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  Size: {size_mb:.1f} MB")
        
        # Cleanup old checkpoints (excluding best_model.pt and final_model.pt)
        if keep_last_n > 0:
            checkpoint_dir = os.path.dirname(path)
            checkpoint_files = sorted([
                f for f in os.listdir(checkpoint_dir)
                if f.startswith('checkpoint_') and f.endswith('.pt')
            ])
            
            # Delete from oldest
            if len(checkpoint_files) > keep_last_n:
                for old_file in checkpoint_files[:-keep_last_n]:
                    old_path = os.path.join(checkpoint_dir, old_file)
                    try:
                        os.remove(old_path)
                        print(f"  Removed old checkpoint: {old_file}")
                    except:
                        pass
    
    def load_checkpoint(self, path: str):
        """Loads checkpoint"""
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        print(f"Checkpoint loaded: {path}")
        return checkpoint['epoch']


# ==================== Main Training Function ====================

def train_dit(
    data_dir: str = "./dit_training_data",
    output_dir: str = "./dit_checkpoints",
    batch_size: int = 4,
    num_epochs: int = 100,
    lr: float = 1e-4,
    save_every: int = 10,
    device: str = 'cuda',
    use_small_model: bool = True  # For disk space optimization
):
    """
    Main training function for DiT
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Dataset
    dataset = TriplaneDataset(
        data_dir=data_dir,
        load_images=True,
        image_size=256,
        triplane_size=256
    )
    
    # Train/Val Split
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Set to 0 to avoid path issues on Windows/Kaggle
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )
    
    # Model Initialization
    if use_small_model:
        # Small Model (~40M params, ~160MB/checkpoint)
        model = DiffusionTransformer(
            img_size=256,
            patch_size=16,
            in_channels=dataset.info['static_channels'] + dataset.info['dynamic_channels'],
            condition_channels=3,
            hidden_size=384,
            depth=6,
            num_heads=6,
            mlp_ratio=4.0
        )
        print("Model: SMALL (40M params, ~160MB)")
    else:
        # Standard Model (~172M params, ~700MB/checkpoint)
        model = DiffusionTransformer(
            img_size=256,
            patch_size=16,
            in_channels=dataset.info['static_channels'] + dataset.info['dynamic_channels'],
            condition_channels=3,
            hidden_size=768,
            depth=12,
            num_heads=12,
            mlp_ratio=4.0
        )
        print("Model: STANDARD (172M params, ~700MB)")
    
    # Diffusion
    diffusion = GaussianDiffusion(
        timesteps=1000,
        beta_start=0.0001,
        beta_end=0.02,
        device=device
    )
    
    # Trainer
    trainer = DiTTrainer(
        model=model,
        diffusion=diffusion,
        device=device,
        lr=lr
    )
    
    # Training Loop
    print(f"\n{'='*60}")
    print("Starting DiT Training")
    print(f"{'='*60}\n")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Batch size: {batch_size}")
    print(f"Epochs: {num_epochs}\n")
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        epoch_loss = 0
        
        for batch in pbar:
            metrics = trainer.train_step(batch)
            epoch_loss += metrics['loss']
            pbar.set_postfix({'loss': f"{metrics['loss']:.4f}"})
        
        avg_train_loss = epoch_loss / len(train_loader)
        
        # Validation
        val_loss = trainer.validate(val_loader)
        
        # Scheduler Step
        trainer.scheduler.step()
        
        print(f"\nEpoch {epoch+1} Complete:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  LR: {trainer.optimizer.param_groups[0]['lr']:.2e}")
        
        # Periodic Save (Keep only last 2)
        if (epoch + 1) % save_every == 0:
            save_path = os.path.join(output_dir, f"checkpoint_{epoch+1:04d}.pt")
            trainer.save_checkpoint(save_path, epoch+1, keep_last_n=2)
        
        # Save Best Model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_path = os.path.join(output_dir, "best_model.pt")
            
            if os.path.exists(best_path):
                os.remove(best_path)
            
            trainer.save_checkpoint(best_path, epoch+1, keep_last_n=0)
            print(f"  ★ Best model saved! (Val Loss: {val_loss:.4f})")
    
    # Cleanup intermediate checkpoints and save final model
    for f in os.listdir(output_dir):
        if f.startswith('checkpoint_') and f.endswith('.pt'):
            try:
                os.remove(os.path.join(output_dir, f))
                print(f"Removed temporary checkpoint: {f}")
            except:
                pass
    
    final_path = os.path.join(output_dir, "final_model.pt")
    trainer.save_checkpoint(final_path, num_epochs, keep_last_n=0)
    
    print(f"\n{'='*60}")
    print("Training Complete!")
    print(f"{'='*60}")
    print(f"Best model: {best_path}")
    print(f"Final model: {final_path}")
    print(f"{'='*60}\n")


if __name__ == "__main__":
    # ========== Kaggle Environment Path Setup ==========
    
    # Input data (Read-only)
    DATA_DIR = "/kaggle/input/cpu-avatarartist3-triplane-decomposition/dit_training_data"
    
    # Output directory (Writable)
    OUTPUT_DIR = "/kaggle/working/dit_checkpoints"
    
    # Path Verification
    print("=" * 60)
    print("Path Verification")
    print("=" * 60)
    print(f"Data Directory: {DATA_DIR}")
    print(f"  Exists: {os.path.exists(DATA_DIR)}")
    
    if os.path.exists(DATA_DIR):
        print(f"  File Count:")
        if os.path.exists(os.path.join(DATA_DIR, 'images')):
            num_images = len(os.listdir(os.path.join(DATA_DIR, 'images')))
            print(f"    images/: {num_images}")
        if os.path.exists(os.path.join(DATA_DIR, 'triplanes')):
            num_triplanes = len(os.listdir(os.path.join(DATA_DIR, 'triplanes')))
            print(f"    triplanes/: {num_triplanes}")
    
    print(f"\nOutput Directory: {OUTPUT_DIR}")
    print(f"  Writable: {os.access('/kaggle/working', os.W_OK)}")
    print("=" * 60 + "\n")
    
    # Check if data directory exists
    if not os.path.exists(DATA_DIR):
        print(f"❌ Data directory not found: {DATA_DIR}")
        print("\nSearching for available datasets...")
        
        kaggle_input = "/kaggle/input"
        if os.path.exists(kaggle_input):
            for dataset in os.listdir(kaggle_input):
                dataset_path = os.path.join(kaggle_input, dataset)
                print(f"\n📁 Dataset: {dataset}")
                
                # Search for dit_training_data within the input datasets
                for root, dirs, files in os.walk(dataset_path):
                    if 'dit_training_data' in root or 'metadata.json' in files:
                        print(f"  Found candidate: {root}")
        
        print("\nPlease update DATA_DIR with one of the paths above.")
        import sys
        sys.exit(1)
    
    # ========== Training Configurations ==========
    
    BATCH_SIZE = 4      # VRAM 24GB: 4-8, 12GB: 2
    NUM_EPOCHS = 80     # Shortened for testing (Prod: 100+)
    LEARNING_RATE = 1e-4
    SAVE_EVERY = 10
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    print(f"Device: {DEVICE}")
    if DEVICE == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB\n")
    
    # Execution
    try:
        train_dit(
            data_dir=DATA_DIR,
            output_dir=OUTPUT_DIR,
            batch_size=BATCH_SIZE,
            num_epochs=NUM_EPOCHS,
            lr=LEARNING_RATE,
            save_every=SAVE_EVERY,
            device=DEVICE
        )
    except Exception as e:
        print(f"\n❌ Training Error: {e}")
        import traceback
        traceback.print_exc()